In [1]:
import pandas as pd
import numpy as np
import ast
import textwrap

import spacy
from bertopic import BERTopic

#Representación de topics
from bertopic.vectorizers import ClassTfidfTransformer
#Embedding model
from sentence_transformers import SentenceTransformer
#UMAP para reducir la dimensionalidad
from umap import UMAP
#clusterizador
from hdbscan import HDBSCAN
#Mapa de texto a vector 
from sklearn.feature_extraction.text import CountVectorizer
#representacion fine-tuned de bert topic: https://maartengr.github.io/BERTopic/index.html#fine-tune-topic-representations
from bertopic.representation import KeyBERTInspired

/home/alejandro/.virtualenvs/mainenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Objetivo

Se tienen unas vacantes laborales para el sector energético. Cada vacante tienen un listado de conocimientos. Se quiere agrupar los conocimientos en categorias estandar para discernir cuales son los conocimientos que está demandando el mercado laboral.

## Referencias

https://maartengr.github.io/BERTopic/index.html#variations

## Funciones

In [2]:
def safe_parse_conocimientos(value):
    '''
    Cambia de formato las entradas de la forma
    "'palabra_1','palabra_2',..., 'palabra_n'"
    a
    ['palabra_1','palabra_2,...,'palabra_n']
    '''
    try:
        # Si no está entre corchetes, lo envolvemos
        if not value.strip().startswith("["):
            value = f"[{value}]"
        return ast.literal_eval(value)
    except Exception:
        return [str(value)]  # Si no se puede evaluar, lo devolvemos como lista con un solo string


## Leer datos

In [3]:
conoc_df=pd.read_excel("../data/conoc_tej.xlsx",index_col=False)
conoc_df=conoc_df.replace('NA',np.nan)

In [4]:
print(f"Numero de filas: {conoc_df.shape[0]} \nNumero de columnas: {conoc_df.shape[1]}")
conoc_df.head()

Numero de filas: 3087 
Numero de columnas: 26


,id,titulo_vacante,descripcion,tipo_documento,cod_depto,fecha_inicio,fecha_cierre,registro_mes,registro_ano,tipo1prestador,...,depto,mpio,rangos_salariales,numero_documento_anom,codigo,descripcion_input,profesion,educacion,conocimientos,clasif_cine_nombre
0,180,auxiliar de servicio tecnico industrial,importante multinacional del sector energetico...,NI,11,44942,30/01/2023,1,2023,Privado,...,"BOGOTA, D. C.",BOGOTA D.C.,$1.500.001 - $2.000.000,fdd3b5bff414026a8d0b7c09bc232281d192797cec6b2b...,OFERTA_181,auxiliar de servicio tecnico industrial; impor...,'Auxiliar servicio técnico industrial',"'Mecánico', 'Electromecánico'","'mantenimiento de maquinaria industrial', 'man...",71.0-Ingeniería y profesiones afines
1,368,tecnico fumigador - con moto,importante empresa dedicada a la prestacion de...,NI,11,44945,30/01/2023,1,2023,Privado,...,"BOGOTA, D. C.",BOGOTA D.C.,$1.000.001 - $1.500.000,fdd3b5bff414026a8d0b7c09bc232281d192797cec6b2b...,OFERTA_369,tecnico fumigador - con moto; importante empre...,'Técnico fumigador','Bachiller',"'Control integrado de plagas', 'Control de roe...",1.0-Programas y certificaciones básicos
2,637,ingeniero preventa,importante empresa requiere para su equipo de ...,NI,11,44950,30/01/2023,1,2023,Privado,...,"BOGOTA, D. C.",BOGOTA D.C.,$2.000.001 - $3.000.000,f1bb3499cfe1370bbf73106827e6ba0c3b6fe5c06db679...,OFERTA_638,ingeniero preventa; importante empresa requier...,'Ingeniero preventa',"'Ingeniería electrónica', 'Mecatrónica', 'Tele...","'Proyectos de seguridad electrónica', 'Dimensi...",41.0-Educación comercial y administración
3,702,tecnico electricista - lugar de trabajo: bogota,importante empresa requiere tecnico electricis...,NI,11,44936,30/01/2023,1,2023,Privado,...,"BOGOTA, D. C.",BOGOTA D.C.,$1.000.001 - $1.500.000,53547f1b0955425b627d74c9865433f6ef9137a8ecb8c0...,OFERTA_703,tecnico electricista - lugar de trabajo: bogot...,'Técnico electricista','NO_APLICA','Mantenimiento preventivo y correctivo de rede...,1.0-Programas y certificaciones básicos
4,1123,vendedor/a - experiencia en ventes telefonicas,importante compania del sector de saneamiento ...,NI,11,44942,17/03/2023,1,2023,Privado,...,"BOGOTA, D. C.",BOGOTA D.C.,$1.000.001 - $1.500.000,e1c11017d0aff28b84b75cef8b7a4d5ae5e5f140530b9c...,OFERTA_1124,vendedor/a - experiencia en ventes telefonicas...,'Vendedor','NO_APLICA',"'Ventas telefónicas', 'Call center'",1.0-Programas y certificaciones básicos


In [5]:
#descripcion de la primera vacante
wrapped_text = textwrap.fill(conoc_df['descripcion'][0], width=80)
print(wrapped_text)

importante multinacional del sector energetico requiere para su equipo de
trabajo tecnico o tecnologo mecanico, electromecanico o a fines, con minimo tres
anos de experiencia en el desarrollo de trabajos asociados con el mantenimiento
de maquinaria industrial (no locativo), manejo de bombas de vacio, equipos de
soldadura, mecanica hidraulica. te ofrecemosii salario:2.000.000 + prestaciones
de ley horario de lunes a sabado - disponibilidad domingo o festivos lugar de
trabajo tenjo cundinamarca contrato obra o labor 12 meses beneficio de casino y
ruta


In [6]:
#Conocimientos de la vacante
conoc_df['conocimientos'][0]

"'mantenimiento de maquinaria industrial', 'manejo de bombas de vacío', 'equipos de soldadura', 'mecánica hidráulica'"

In [7]:
#Generamos una lista con los conocimientos individuales
#En esta version, cada documento es un conocimiento especifico
corpus = conoc_df['conocimientos'].apply(safe_parse_conocimientos).sum()
print(f"Numero de documentos en el corpus:{len(corpus)}")
#En esta version, cada "documento" es un listado de conocimientos
#conocimientos = conoc_df.conocimientos.to_list()

Numero de documentos en el corpus:16508


## Instanciar modelo

In [8]:
#spamsih stop words
nlp = spacy.blank("es")
spanish_stopwords = nlp.Defaults.stop_words
del(nlp)

c-TF-IDF will show you what words are most important for each topic, relative to the other topics in the corpus. This allows you to identify topic-specific terms that help characterize the content of each topic.

In [11]:
#Extract embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
#UMAP para reducir la dimensionalidad de los embeddings
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=123)
#Clusterizador de emebeddings
hdbscan_model = HDBSCAN(min_cluster_size=50, metric='sine', cluster_selection_method='eom', prediction_data=True)
#Tokenizador de topicos
vectorizer_model = CountVectorizer(stop_words=list(spanish_stopwords), min_df=2, ngram_range=(1,3))
#Create topic representation
ctfidf_model = ClassTfidfTransformer()
#representacion fine-tuned de bert topic:
representation_model = KeyBERTInspired()


model = BERTopic(
    embedding_model=embedding_model,            # Step 1 - Extract embeddings
    umap_model=umap_model,                      # Step 2 - Reduce dimensionality
    hdbscan_model=hdbscan_model,                # Step 3 - Cluster reduced embeddings
    vectorizer_model=vectorizer_model,          # Step 4 - Tokenize topics
    ctfidf_model=ctfidf_model,                  # Step 5 - Extract topic words
    representation_model=representation_model,  # Step 6 - (Optional) Fine-tune topic representations
    language= "spanish"
    )#"spanish"#"multilingual


#model = BERTopic(vectorizer_model=vectorizer_model,umap_model=umap_model, representation_model=representation_model,language= "spanish")#"spanish"#"multilingual

topics, probabilities = model.fit_transform(corpus)


InvalidParameterError: The 'metric' parameter of pairwise_distances must be a str among {'haversine', 'l2', 'canberra', 'braycurtis', 'wminkowski', 'matching', 'nan_euclidean', 'sokalmichener', 'chebyshev', 'correlation', 'rogerstanimoto', 'cosine', 'precomputed', 'mahalanobis', 'sqeuclidean', 'yule', 'euclidean', 'manhattan', 'russellrao', 'seuclidean', 'jaccard', 'hamming', 'minkowski', 'dice', 'l1', 'sokalsneath', 'cityblock'} or a callable. Got 'sine' instead.

In [ ]:
print(f"Numero de topicos: {len(model.get_topics())}")
print("Ejemplo te topico:\n")

Numero de topicos: 94
Ejemplo te topico:



[('ambientales seguimiento', np.float32(0.88842607)),
 ('ambientales licenciamiento', np.float32(0.88002324)),
 ('estudios ambientales', np.float32(0.87770194)),
 ('ambientales', np.float32(0.8560213)),
 ('elaboración estudios ambientales', np.float32(0.8471186)),
 ('ambiental seguimiento', np.float32(0.83231235)),
 ('ambiental elaboración', np.float32(0.8288516)),
 ('protección ambiental', np.float32(0.8222429)),
 ('licencias ambientales', np.float32(0.81547165)),
 ('ambiental proyectos', np.float32(0.79367656))]

In [26]:
#outliers
print("Outliers:\n")
model.get_topic( -1 )

Outliers:



[('paquete office', np.float32(0.9078065)),
 ('paquete', np.float32(0.7176398)),
 ('office', np.float32(0.5236668)),
 ('soporte', np.float32(0.48858148)),
 ('soporte técnico', np.float32(0.46840516)),
 ('sistemas', np.float32(0.46502215)),
 ('corte', np.float32(0.45178977)),
 ('sistema', np.float32(0.44529915)),
 ('pruebas', np.float32(0.43587822)),
 ('sistemas integrados gestión', np.float32(0.41668606))]

In [ ]:
#reduced_topics=model.reduce_outliers(corpus,topics)
#model.update_topics(corpus,topics=reduced_topics)

# Visualizar

In [28]:
model.visualize_topics()

In [29]:
model.visualize_barchart()

# TBD

In [40]:
documentos=model.get_document_info(corpus)

In [43]:
documentos.head()

,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document
0,mantenimiento de maquinaria industrial,2,2_mantenimiento redes eléctricas_mantenimiento...,"[mantenimiento redes eléctricas, mantenimiento...","[Mantenimiento de redes eléctricas, Mantenimie...",mantenimiento redes eléctricas - mantenimiento...,0.060386,False
1,manejo de bombas de vacío,14,14_tratamiento aguas_tratamiento agua_sistemas...,"[tratamiento aguas, tratamiento agua, sistemas...","[Control de plantas de tratamiento de agua, Pl...",tratamiento aguas - tratamiento agua - sistema...,0.208504,False
2,equipos de soldadura,-1,-1_paquete office_paquete_office_soporte,"[paquete office, paquete, office, soporte, sop...","[Paquete Office, Paquete Office, Paquete Office]",paquete office - paquete - office - soporte - ...,0.000000,False
3,mecánica hidráulica,-1,-1_paquete office_paquete_office_soporte,"[paquete office, paquete, office, soporte, sop...","[Paquete Office, Paquete Office, Paquete Office]",paquete office - paquete - office - soporte - ...,0.000000,False
4,Control integrado de plagas,-1,-1_paquete office_paquete_office_soporte,"[paquete office, paquete, office, soporte, sop...","[Paquete Office, Paquete Office, Paquete Office]",paquete office - paquete - office - soporte - ...,0.000000,False


# Clasificación Jerárquica

In [44]:
from scipy.cluster import hierarchy as sch

In [ ]:
linkage_function = lambda x: sch.linkage(x, 'single', optimal_ordering=True)
model_jer=model.hierarchical_topics(corpus, linkage_function = linkage_function)    

100%|██████████| 92/92 [00:15<00:00,  6.11it/s]


In [46]:
model.visualize_hierarchy(hierarchical_topics=model_jer)

In [ ]:
tree = model.get_topic_tree(model_jer)
>>> print(tree)

.
├─soporte_apoyar_soporte soporte_llamada soporte_itil soporte
│    ├─■──soporte_apoyar_soporte reas_itil soporte_soporte soporte ── Topic: 5
│    └─■──llamada soporte_llamada_soporte soporte_soporte_soporte soporte soporte ── Topic: 168
└─servicio cliente_cliente_servicios_servicio_server
     ├─servicio cliente_servicios_servicio_cliente_server
     │    ├─■──tecnicos_operarios_especializados_tecnicos tecnicos_operativos ── Topic: 151
     │    └─server_plataformas_informatica_servicio cliente_microsoft
     │         ├─■──computador_internet_camara_soporte_ai ── Topic: 81
     │         └─server_servicio cliente_microsoft_informatica_linux
     │              ├─■──ejercicio_recreacion_deportes_deportiva_deporte ── Topic: 94
     │              └─informatica_servicio cliente_server_microsoft_servicios
     │                   ├─■──administracion operacion equipos_operacion equipos_equipos procesos_procesos equipos_operacion maqui ── Topic: 64
     │                   └─informatica_m

In [39]:
model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,5745,-1_paquete office_paquete_office_soporte,"[paquete office, paquete, office, soporte, sop...","[Paquete Office, Paquete Office, Paquete Office]"
1,0,644,0_sistemas eléctricos_eléctricos sistemas_eléc...,"[sistemas eléctricos, eléctricos sistemas, elé...","[Sistemas eléctricos, Sistemas eléctricos, Sis..."
2,1,445,1_ambientales seguimiento_ambientales licencia...,"[ambientales seguimiento, ambientales licencia...","[Ciencias ambientales, Monitoreos ambientales,..."
3,2,413,2_mantenimiento redes eléctricas_mantenimiento...,"[mantenimiento redes eléctricas, mantenimiento...","[Mantenimiento de redes eléctricas, Mantenimie..."
4,3,393,3_software_software diseño_programadas_softwar...,"[software, software diseño, programadas, softw...","[Software, Software HECHMS, Software de Office]"
...,...,...,...,...,...
89,88,53,88_aplicativos___,"[aplicativos, , , , , , , , , ]","[NO_APLICA, NO_APLICA, Aplicativos]"
90,89,53,89_residuos hospitalarios_hospitalarios manejo...,"[residuos hospitalarios, hospitalarios manejo,...","[Manejo de residuos hospitalarios, Manejo de r..."
91,90,53,90_contratación_seguimiento contratos_selecció...,"[contratación, seguimiento contratos, selecció...","[Planeación de compras y contratación, Planeac..."
92,91,53,91_diseño_diseno_disenos_diseños,"[diseño, diseno, disenos, diseños, mecánico, ,...","[Diseño, Diseño, Diseño]"


In [81]:
documentos

,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document
0,mantenimiento de maquinaria industrial,-1,-1_servicio al cliente_en el trabajo_servicio ...,"[servicio al cliente, en el trabajo, servicio ...","[Huella de carbono, Sistema de gestión de segu...",servicio al cliente - en el trabajo - servicio...,0.000000,False
1,manejo de bombas de vacío,-1,-1_servicio al cliente_en el trabajo_servicio ...,"[servicio al cliente, en el trabajo, servicio ...","[Huella de carbono, Sistema de gestión de segu...",servicio al cliente - en el trabajo - servicio...,0.000000,False
2,equipos de soldadura,-1,-1_servicio al cliente_en el trabajo_servicio ...,"[servicio al cliente, en el trabajo, servicio ...","[Huella de carbono, Sistema de gestión de segu...",servicio al cliente - en el trabajo - servicio...,0.000000,False
3,mecánica hidráulica,75,75_instalación tubería_sistemas seguridad elec...,"[instalación tubería, sistemas seguridad elect...",[Instalación de tubería y cableado para sistem...,instalación tubería - sistemas seguridad elect...,1.000000,False
4,Control integrado de plagas,-1,-1_servicio al cliente_en el trabajo_servicio ...,"[servicio al cliente, en el trabajo, servicio ...","[Huella de carbono, Sistema de gestión de segu...",servicio al cliente - en el trabajo - servicio...,0.000000,False
...,...,...,...,...,...,...,...,...
16503,Soporte técnico,16,16_soporte tecnico_soporte técnico_soporte téc...,"[soporte tecnico, soporte técnico, soporte téc...","[Soporte técnico, Soporte técnico, Soporte téc...",soporte tecnico - soporte técnico - soporte té...,0.837394,True
16504,Mantenimiento preventivo,-1,-1_servicio al cliente_en el trabajo_servicio ...,"[servicio al cliente, en el trabajo, servicio ...","[Huella de carbono, Sistema de gestión de segu...",servicio al cliente - en el trabajo - servicio...,0.000000,False
16505,Sistemas de refrigeración,-1,-1_servicio al cliente_en el trabajo_servicio ...,"[servicio al cliente, en el trabajo, servicio ...","[Huella de carbono, Sistema de gestión de segu...",servicio al cliente - en el trabajo - servicio...,0.000000,False
16506,Sistemas HVAC,81,81_hvac_hvac proyectos_sistemas aire acondicio...,"[hvac, hvac proyectos, sistemas aire acondicio...","[Sistemas HVAC, Sistemas HVAC, Sistemas HVAC]",hvac - hvac proyectos - sistemas aire acondici...,1.000000,True


In [87]:
documentos = model.get_document_info(conocimientos_list)
print(f"Numero de filas: {documentos.shape[0]} \nNumero de columnas: {documentos.shape[1]}")
documentos.head()


Numero de filas: 16508 
Numero de columnas: 8


,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document
0,mantenimiento de maquinaria industrial,-1,-1_servicio al cliente_en el trabajo_servicio ...,"[servicio al cliente, en el trabajo, servicio ...","[Huella de carbono, Sistema de gestión de segu...",servicio al cliente - en el trabajo - servicio...,0.0,False
1,manejo de bombas de vacío,-1,-1_servicio al cliente_en el trabajo_servicio ...,"[servicio al cliente, en el trabajo, servicio ...","[Huella de carbono, Sistema de gestión de segu...",servicio al cliente - en el trabajo - servicio...,0.0,False
2,equipos de soldadura,-1,-1_servicio al cliente_en el trabajo_servicio ...,"[servicio al cliente, en el trabajo, servicio ...","[Huella de carbono, Sistema de gestión de segu...",servicio al cliente - en el trabajo - servicio...,0.0,False
3,mecánica hidráulica,75,75_instalación tubería_sistemas seguridad elec...,"[instalación tubería, sistemas seguridad elect...",[Instalación de tubería y cableado para sistem...,instalación tubería - sistemas seguridad elect...,1.0,False
4,Control integrado de plagas,-1,-1_servicio al cliente_en el trabajo_servicio ...,"[servicio al cliente, en el trabajo, servicio ...","[Huella de carbono, Sistema de gestión de segu...",servicio al cliente - en el trabajo - servicio...,0.0,False


In [ ]:
documentos['id'] = list(conoc_df['id'])
documentos['descripcion'] =list(conoc_df['descripcion'])
documentos['conocimientos'] =list(conoc_df['conocimientos'])
documentos['titulo_vacante'] =list(conoc_df['titulo_vacante'])

In [ ]:
documentos.head()

In [ ]:
#guardar los resultados en la carpeta output
documentos.to_excel("../output/doc_topics_tec_anom.xlsx")

In [ ]:
documentos.head()

,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document,id,conocimientos,titulo_vacante,descripcion
0,"'mantenimiento de maquinaria industrial', 'man...",25,25_maquinaria_industrial_vacío_hidráulica,"[maquinaria, industrial, vacío, hidráulica, bo...","['Mantenimiento de maquinaria industrial', 'Bo...",maquinaria - industrial - vacío - hidráulica -...,0.779500,False,180,"'mantenimiento de maquinaria industrial', 'man...",auxiliar de servicio tecnico industrial,importante multinacional del sector energetico...
1,"'Control integrado de plagas', 'Control de roe...",28,28_plagas_lavado_limpieza_desinfección,"[plagas, lavado, limpieza, desinfección, tanqu...","['Lavado de tanques', 'Curso de alturas', 'Man...",plagas - lavado - limpieza - desinfección - ta...,1.000000,False,368,"'Control integrado de plagas', 'Control de roe...",tecnico fumigador - con moto,importante empresa dedicada a la prestacion de...
2,"'Proyectos de seguridad electrónica', 'Dimensi...",27,27_cctv_acceso_cableado_sistemas,"[cctv, acceso, cableado, sistemas, incendios, ...","['Sistemas UPS', 'Cableado eléctrico', 'Sistem...",cctv - acceso - cableado - sistemas - incendio...,0.000000,False,637,"'Proyectos de seguridad electrónica', 'Dimensi...",ingeniero preventa,importante empresa requiere para su equipo de ...
3,'Mantenimiento preventivo y correctivo de rede...,41,41_eléctricas_redes_instalación_mantenimiento,"[eléctricas, redes, instalación, mantenimiento...","['Mantenimiento preventivo redes eléctricas', ...",eléctricas - redes - instalación - mantenimien...,0.995023,False,702,'Mantenimiento preventivo y correctivo de rede...,tecnico electricista - lugar de trabajo: bogota,importante empresa requiere tecnico electricis...
4,"'Ventas telefónicas', 'Call center'",5,5_clientes_ventas_venta_comercial,"[clientes, ventas, venta, comercial, productos...","['Ventas', 'Gestión de clientes', 'Negociacion...",clientes - ventas - venta - comercial - produc...,0.639780,False,1123,"'Ventas telefónicas', 'Call center'",vendedor/a - experiencia en ventes telefonicas,importante compania del sector de saneamiento ...


In [ ]:
dendo=model.visualize_hierarchy(hierarchical_topics=model_jer)

In [ ]:
pip install -U kaleido

In [ ]:
pip install --force-reinstall -v "kaleido==0.1.0"

In [ ]:
import plotly.graph_objects as go

In [ ]:
fig = go.Figure(dendo)

In [ ]:
fig

In [ ]:
import plotly.express as px

In [ ]:
!conda install -c conda-forge python-kaleido=0.1.0

In [ ]:
dendo.write_image("C:/Users/USER/Documents/ATENEA/VACANTES/TEC/3_analisis_agrupa/output/dendo_dep.pdf")